In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models

In [29]:
VGG19 = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'] # FCS 4096*4096*1000

In [30]:
class VGG_NN(nn.Module):
    def __init__(self, in_channels, num_classes=1000):
        super(VGG_NN, self).__init__()
        self.in_channels = in_channels
        self.conv_layers = self.create_conv_layers(VGG19)
        self.fcs = nn.Sequential(
            nn.Linear(512*7*7, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(4096, num_classes)
        )
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.reshape(x.shape[0], -1)
        self.feautures = x
        x = self.fcs(x)
        return x
    
    def create_conv_layers(self, architecture):
        layers = []
        in_channels = self.in_channels
        
        for x in architecture:
            if type(x) == int:
                out_channels = x
                layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels, 
                                     kernel_size=(3,3), stride=(1,1), padding=(1,1)),
                           nn.BatchNorm2d(x),
                           nn.ReLU()]
                in_channels = x
            else:
                layers += [nn.MaxPool2d(kernel_size=(2, 2), stride=(2,2))]
        
        return nn.Sequential(*layers)

In [37]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [38]:
model = VGG_NN(in_channels=3, num_classes=1000).to(device)
x = torch.randn(1, 3, 224, 224).to(device)
print(model(x).shape)

torch.Size([1, 1000])
